In [72]:
import json
from kafka import KafkaProducer

# Kafka Producer 설정
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    batch_size=100,
    linger_ms=10,
    acks=1
)

In [73]:
import pytz
import random
import uuid
import pandas as pd
import datetime

sent_events = []

# 1. 이벤트 시간의 시작 지점 설정 (한국 시간 기준으로 설정)
# 예시: 현재 시각으로부터 2일 전, 오전 9시 정각
korea_tz = pytz.timezone('Asia/Seoul')
start_time_kst = korea_tz.localize(datetime.datetime(2025, 6, 28, 11, 26, 0, 0)) # 연, 월, 일, 시, 분, 초, 마이크로초

# 2. 각 이벤트마다 증가시킬 시간 간격 설정
# 예시: 각 이벤트마다 10초씩 증가 (테스트를 위해 짧게 설정)
time_increment = datetime.timedelta(seconds=90)

user_ids = ["111", "222", "333"]
num_events_per_user = 4 # 각 사용자당 보낼 이벤트 개수

current_event_time = start_time_kst

print(f"Starting event time generation from: {current_event_time.isoformat()}")

for _ in range(num_events_per_user * len(user_ids)): # 총 이벤트 개수만큼 반복
    # 사용자 ID를 순환하며 할당
    current_user_id = random.choice(user_ids) # user_ids[i % len(user_ids)] 등으로 순차 할당도 가능

    event = {
        "event_time": current_event_time.isoformat(), # ISO 8601 포맷으로 변환 (예: 2025-06-28T09:00:00+09:00)
        "event_type": random.choice(["view", "cart", "purchase"]),
        "product_id": str(random.randint(10000000, 99999999)),
        "category_id": str(random.randint(1000000000000000000, 9999999999999999999)),
        "category_code": random.choice(["electronics", "beauty", None]),
        "brand": random.choice(["apple", "samsung", "nike"]),
        "price": round(random.uniform(10, 500), 2),
        "user_id": current_user_id,
        "user_session": str(uuid.uuid4())
    }

    # 카프카로 이벤트 전송 (실제 사용 시 주석 해제)
    producer.send("ecommerce-events", value=event)
    sent_events.append(event)

    # 다음 이벤트를 위해 시간 증가
    current_event_time += time_increment

producer.flush() # 모든 메시지가 전송될 때까지 대기
sent_df = pd.DataFrame(sent_events)

Starting event time generation from: 2025-06-28T11:26:00+09:00


In [74]:
# DataFrame 전체 표시 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 전송된 데이터 전체 표시
display(sent_df)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2025-06-28T11:26:00+09:00,purchase,39587389,6297065380356812064,electronics,apple,181.74,333,29ce9d8e-f5d7-4640-b801-046434c350c3
1,2025-06-28T11:27:30+09:00,cart,91421958,4970666548362557025,None,nike,84.19,111,39f05fb3-ada4-45fc-8937-c74209e3297b
2,2025-06-28T11:29:00+09:00,purchase,52482067,3611812611301764579,None,nike,268.27,333,6d3106e1-b157-4c4e-ac77-6b40fc33de23
3,2025-06-28T11:30:30+09:00,cart,64340016,5674015680113821981,None,apple,218.85,111,65366c14-9ce8-49d0-85d5-4318a064f9e8
4,2025-06-28T11:32:00+09:00,view,51405303,4273731759773378137,electronics,nike,171.55,222,230f61f0-6dc9-4aa8-a005-2fef462f143a
5,2025-06-28T11:33:30+09:00,cart,69111246,3776961417575704303,electronics,apple,433.65,111,b0ca5475-64b1-4e18-ad5a-39e44a45b296
6,2025-06-28T11:35:00+09:00,purchase,59323892,7940033754886659056,beauty,apple,167.91,111,b2d5dd5c-1775-4885-82b6-be5d6f7d32e0
7,2025-06-28T11:36:30+09:00,view,67414918,4488729635699100882,None,samsung,351.11,333,aaa751e3-5ee6-438c-8742-3c8524e5dd41
8,2025-06-28T11:38:00+09:00,cart,85062053,5366174719941124986,beauty,samsung,99.01,111,7e07c092-a6f8-4462-b28a-93df8fda5dca
9,2025-06-28T11:39:30+09:00,purchase,11774150,1950185599118265585,electronics,samsung,58.95,111,b1279a2a-37e2-4e62-a919-736dd1c72487


In [75]:
# Redis 연결 및 키 확인
import redis
import json
from pprint import pprint

# Redis 클라이언트 생성
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

try:
    r.ping()
    print('Redis 연결 성공')
except Exception as e:
    print(f"Redis 연결 실패: {e}")


Redis 연결 성공


In [77]:
# 1. 모든 user_features 키 조회
user_keys = r.keys("*")
for key in user_keys[:10]:  # 처음 10개만 표시
    print(f"- {key}")


In [ ]:
r.hgetall("user_features:222")

{}